# path

# import

In [2]:
import os
import pandas as pd
import pymupdf
import pymongo
from pymongo import MongoClient
import qdrant_client
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel


/home/jacktran/.cache/pypoetry/virtualenvs/testing-yfdd-Kz9-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## connect to database


In [3]:

try:
    client = MongoClient(DATABASE_HOST)
    client.admin.command("ping")
    print("connected successfully")
except Exception as e:
    raise Exception("The following error occurred: ", e)

connected successfully


In [4]:
database = client["RAG"]
collection = database["extracted_pdfs"]

# extract

In [12]:
def extract_pdf(pdf_path, target_path):
    doc = pymupdf.open(pdf_path)
    with open(target_path, "wb") as out:
        for page in doc:
            text = page.get_text().encode("utf-8")
            out.write(text)
            out.write(bytes((12,)))

In [13]:
def extract_all_in_directory(source: str, target: str) -> None:
    if not os.path.exists(target):
        os.mkdir(target)

    for file in os.listdir(source):
        if file.endswith(".pdf"):
            file_path = os.path.join(source, file)
            target_file= file.split(".")[0] + ".txt"
            target_file_path = os.path.join(target, target_file)
            extract_pdf(file_path, target_file_path)
            print(f"extract from {file_path} to {target_file_path}")

In [14]:
extract_all_in_directory(source_directory, target_directory)

extract from /home/jacktran/RAG/experiment/files/927ac79f-30b6-420c-af2f-9229ba129bdf.pdf to /home/jacktran/RAG/experiment/target_files/927ac79f-30b6-420c-af2f-9229ba129bdf.txt
extract from /home/jacktran/RAG/experiment/files/KBSV_Baocaocapnhat_MBB_092025..pdf to /home/jacktran/RAG/experiment/target_files/KBSV_Baocaocapnhat_MBB_092025.txt
extract from /home/jacktran/RAG/experiment/files/SSI_Q3_25_a82298dac6.pdf to /home/jacktran/RAG/experiment/target_files/SSI_Q3_25_a82298dac6.txt
extract from /home/jacktran/RAG/experiment/files/c883c871-d5d1-404f-b56a-e44322d36843.pdf to /home/jacktran/RAG/experiment/target_files/c883c871-d5d1-404f-b56a-e44322d36843.txt
extract from /home/jacktran/RAG/experiment/files/Baocaocapnhat_VPB_Q3_VN.pdf to /home/jacktran/RAG/experiment/target_files/Baocaocapnhat_VPB_Q3_VN.txt
extract from /home/jacktran/RAG/experiment/files/5fa639a7-4e66-495c-9e6f-22732ac748f4.pdf to /home/jacktran/RAG/experiment/target_files/5fa639a7-4e66-495c-9e6f-22732ac748f4.txt
extract f

In [17]:
def insert_all_txt(folder_path):
    docs = []

    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)

            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            docs.append({
                "file_name": file,
                "text": content
            })

    if docs:
        collection.insert_many(docs)

insert_all_txt("/home/jacktran/RAG/experiment/target_files")

# Cleaning

In [19]:
def cleaning(serie: pd.Series) -> pd.Series:
    return (
        serie
        .str.replace(r'\n{2,}', '. ', regex=True)
        .str.replace(r'[ \t\r\n]+', ' ', regex=True)
        .str.replace(' . ', '. ') 
        .str.strip()      
    )

In [22]:
docs = list(collection.find({}))
df = pd.DataFrame(docs)
df.head()

,_id,file_name,text
0,699eb2d532eea16500c0828b,PVD_271125_ACBS27112025160032.txt,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...
1,699eb2d532eea16500c0828c,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026:\n% tăng giá:\nCập nhật:\nTR...
2,699eb2d532eea16500c0828d,KBSV_Baocaocapnhat_MBB_092025.txt,Ngân hàng TMCP Quân đội (MBB) \nChuyển biến tí...
3,699eb2d532eea16500c0828e,c883c871-d5d1-404f-b56a-e44322d36843.txt,\n \nBỘ GIÁO DỤC VÀ ĐÀO TẠO \nTRƯỜNG ĐẠI HỌC ...
4,699eb2d532eea16500c0828f,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CÔ...


In [23]:
df['text'] = cleaning(df['text'])
df.head()

,_id,file_name,text
0,699eb2d532eea16500c0828b,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...
1,699eb2d532eea16500c0828c,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...
2,699eb2d532eea16500c0828d,KBSV_Baocaocapnhat_MBB_092025.txt,Ngân hàng TMCP Quân đội (MBB) Chuyển biến tích...
3,699eb2d532eea16500c0828e,c883c871-d5d1-404f-b56a-e44322d36843.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...
4,699eb2d532eea16500c0828f,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...


In [5]:
cleaned_collection = database["cleaned_pdfs"]

In [25]:
df_cleaned = df.drop(columns=["_id"], errors="ignore")
cleaned_collection.insert_many(df_cleaned.to_dict("records"))

InsertManyResult([ObjectId('699eba5632eea16500c08296'), ObjectId('699eba5632eea16500c08297'), ObjectId('699eba5632eea16500c08298'), ObjectId('699eba5632eea16500c08299'), ObjectId('699eba5632eea16500c0829a'), ObjectId('699eba5632eea16500c0829b'), ObjectId('699eba5632eea16500c0829c'), ObjectId('699eba5632eea16500c0829d'), ObjectId('699eba5632eea16500c0829e'), ObjectId('699eba5632eea16500c0829f'), ObjectId('699eba5632eea16500c082a0')], acknowledged=True)

# Chunking

In [6]:
cleaned_docs = list(cleaned_collection.find({}))
df_chunking = pd.DataFrame(cleaned_docs)
df_chunking = df_chunking.drop(columns=["_id"], errors='ignore')
df_chunking.head()

,file_name,text
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...
1,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...
2,KBSV_Baocaocapnhat_MBB_092025.txt,Ngân hàng TMCP Quân đội (MBB) Chuyển biến tích...
3,c883c871-d5d1-404f-b56a-e44322d36843.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...
4,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

df_chunking['n_tokens'] = df_chunking['text'].apply(lambda x: len(tokenizer.encode(x)))


/home/jacktran/.cache/pypoetry/virtualenvs/testing-yfdd-Kz9-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (6434 > 512). Running this sequence through the model will result in indexing errors


In [8]:
df_chunking.head()

,file_name,text,n_tokens
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,6434
1,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...,2616
2,KBSV_Baocaocapnhat_MBB_092025.txt,Ngân hàng TMCP Quân đội (MBB) Chuyển biến tích...,7312
3,c883c871-d5d1-404f-b56a-e44322d36843.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...,5659
4,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...,855


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def chunking_with_metadata(file_name, full_text, max_token=500):
    pages = full_text.split('\x0c')
    chunks_with_metadata = []
    current_chunk_sentences = []
    current_tokens = 0
    start_page = 1 

    for i, page_content in enumerate(pages):
        page_num = i + 1
        sentences = [s.strip() for s in page_content.split('. ') if s.strip()]
        
        for sent in sentences:
            sent_with_dot = sent + ". "
            token_ids = tokenizer.encode(sent_with_dot, add_special_tokens=False, truncation=True, max_length=1024)
            token_len = len(token_ids)
            
            if token_len > max_token:
                if current_chunk_sentences:
                    chunks_with_metadata.append({
                        'file': file_name,
                        'text': " ".join(current_chunk_sentences).strip(),
                        'page': f"{start_page}-{page_num}" if start_page != page_num else page_num
                    })
                    current_chunk_sentences = []
                    current_tokens = 0

                sub_chunks = [token_ids[i:i + max_token] for i in range(0, token_len, max_token)]
                for sub in sub_chunks:
                    chunks_with_metadata.append({
                        'file': file_name,
                        'text': tokenizer.decode(sub),
                        'page': page_num
                    })
                start_page = page_num
                continue

            if current_tokens + token_len > max_token:
                chunks_with_metadata.append({
                    'file': file_name,
                    'text': " ".join(current_chunk_sentences).strip(),
                    'page': f"{start_page}-{page_num}" if start_page != page_num else page_num
                })
                
                last_sentence = current_chunk_sentences[-1] if current_chunk_sentences else ""
                current_chunk_sentences = [last_sentence, sent] if last_sentence else [sent]
                current_tokens = len(tokenizer.encode(last_sentence + ". " + sent_with_dot, add_special_tokens=False, truncation=True))
                start_page = page_num
            else:
                current_chunk_sentences.append(sent)
                current_tokens += token_len

    if current_chunk_sentences:
        chunks_with_metadata.append({
            'file': file_name, 
            'text': " ".join(current_chunk_sentences).strip(),
            'page': f"{start_page}-{len(pages)}" if start_page != len(pages) else start_page
        })
        
    return chunks_with_metadata


all_records = []
for _, row in df_chunking.iterrows():
    if row['text']:
        res = chunking_with_metadata(row['file_name'], row['text'])
        all_records.extend(res)

df_vector = pd.DataFrame(all_records)

In [10]:
df_vector.head()

,file,text,page
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,1-2
1,PVD_271125_ACBS27112025160032.txt,"Bên cạnh đó, khối lượng công việc của mảng Kỹ ...",2
2,PVD_271125_ACBS27112025160032.txt,Trương Anh Quốc (+84 8) 7300 7000 - Ext: 1044 ...,2
3,PVD_271125_ACBS27112025160032.txt,0.9 0.8 Cổ tức (đồng) 3 3 3 2 2 Suất sinh lợi ...,2
4,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,2-3


In [11]:
df_vector['n_tokens'] = df_vector.text.apply(lambda x: len(tokenizer.encode(x)))
df_vector

Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors


,file,text,page,n_tokens
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,1-2,484
1,PVD_271125_ACBS27112025160032.txt,"Bên cạnh đó, khối lượng công việc của mảng Kỹ ...",2,256
2,PVD_271125_ACBS27112025160032.txt,Trương Anh Quốc (+84 8) 7300 7000 - Ext: 1044 ...,2,502
3,PVD_271125_ACBS27112025160032.txt,0.9 0.8 Cổ tức (đồng) 3 3 3 2 2 Suất sinh lợi ...,2,64
4,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,2-3,453
...,...,...,...,...
105,Baocaocapnhat_VPB_Q3_VN.txt,Trang 9 Ngân hàng TMCP Việt Nam Thịnh Vượng (H...,9-10,400
106,Baocaocapnhat_VPB_Q3_VN.txt,Trang 10 Ngân hàng TMCP Việt Nam Thịnh Vượng (...,10,502
107,Baocaocapnhat_VPB_Q3_VN.txt,+286 đcb CIR 23.0% -499 đcb 22.0% -291 đcb -46...,10,72
108,Baocaocapnhat_VPB_Q3_VN.txt,Trang 11 Ngân hàng TMCP Việt Nam Thịnh Vượng (...,10-12,478


In [12]:
df_vector['n_tokens'].max()

np.int64(538)

# Embedding

## dense vector

In [14]:
model = SentenceTransformer(MODEL_ID)

In [15]:
def get_embedding(text: str):
    text = text.replace("\n", " ")
    return model.encode(text, normalize_embeddings=True).tolist()

df_vector["embeddings"] = df_vector["text"].apply(get_embedding)

In [16]:
df_vector

,file,text,page,n_tokens,embeddings
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,1-2,484,"[0.01686115376651287, 0.03231779485940933, -0...."
1,PVD_271125_ACBS27112025160032.txt,"Bên cạnh đó, khối lượng công việc của mảng Kỹ ...",2,256,"[0.00847559329122305, 0.01953323744237423, -0...."
2,PVD_271125_ACBS27112025160032.txt,Trương Anh Quốc (+84 8) 7300 7000 - Ext: 1044 ...,2,502,"[0.007613793481141329, 0.02797575481235981, -0..."
3,PVD_271125_ACBS27112025160032.txt,0.9 0.8 Cổ tức (đồng) 3 3 3 2 2 Suất sinh lợi ...,2,64,"[0.019091611728072166, 0.0036112633533775806, ..."
4,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,2-3,453,"[0.020067453384399414, 0.027360811829566956, -..."
...,...,...,...,...,...
105,Baocaocapnhat_VPB_Q3_VN.txt,Trang 9 Ngân hàng TMCP Việt Nam Thịnh Vượng (H...,9-10,400,"[0.01837308146059513, 0.0036350220907479525, -..."
106,Baocaocapnhat_VPB_Q3_VN.txt,Trang 10 Ngân hàng TMCP Việt Nam Thịnh Vượng (...,10,502,"[0.02595464698970318, 0.005091124214231968, -0..."
107,Baocaocapnhat_VPB_Q3_VN.txt,+286 đcb CIR 23.0% -499 đcb 22.0% -291 đcb -46...,10,72,"[0.0014486246509477496, 0.005260942969471216, ..."
108,Baocaocapnhat_VPB_Q3_VN.txt,Trang 11 Ngân hàng TMCP Việt Nam Thịnh Vượng (...,10-12,478,"[0.015954485163092613, 0.01128364633768797, -0..."


In [17]:
import numpy as np

embeddings = np.array(df_vector["embeddings"].tolist())
embeddings.shape

(110, 1024)

## sparse vector

In [ ]:

sparse_model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 107638.25it/s]


In [ ]:
def get_sparse(texts):
    output = sparse_model.encode(
        texts,
        return_dense=False,         
        return_sparse=True,          
        return_colbert_vecs=False
    )
    return output["lexical_weights"]

texts = ["Tôi đang xây dựng hệ thống RAG bằng Qdrant"]
sparse_vectors = get_sparse(texts)

print(sparse_vectors[0])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


defaultdict(<class 'int'>, {'14343': np.float16(0.1301), '4724': np.float16(0.1201), '14948': np.float16(0.1653), '13291': np.float16(0.1354), '7099': np.float16(0.1527), '10657': np.float16(0.1373), '19400': np.float16(0.2069), '724': np.float16(0.239), '6567': np.float16(0.1282), '2396': np.float16(0.1431), '71': np.float16(0.1183), '15212': np.float16(0.3057)})


In [20]:
def get_sparse_embedding(text: str):
    text = text.replace("\n", " ")
    
    output = sparse_model.encode(
        [text],
        return_dense=False,
        return_sparse=True,
        return_colbert_vecs=False
    )
    
    sparse_dict = output["lexical_weights"][0]
    
    return {
        "indices": [int(k) for k in sparse_dict.keys()],
        "values": [float(v) for v in sparse_dict.values()]
    }

In [21]:
df_vector["sparse"] = df_vector["text"].apply(get_sparse_embedding)

In [26]:
df_vector.head()

,file,text,page,n_tokens,embeddings,sparse
0,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,1-2,484,"[0.01686115376651287, 0.03231779485940933, -0....","{'indices': [204101, 42897, 82375, 397, 46, 70..."
1,PVD_271125_ACBS27112025160032.txt,"Bên cạnh đó, khối lượng công việc của mảng Kỹ ...",2,256,"[0.00847559329122305, 0.01953323744237423, -0....","{'indices': [26986, 94664, 6372, 1871, 2735, 1..."
2,PVD_271125_ACBS27112025160032.txt,Trương Anh Quốc (+84 8) 7300 7000 - Ext: 1044 ...,2,502,"[0.007613793481141329, 0.02797575481235981, -0...","{'indices': [181005, 9735, 8735, 22183, 13903,..."
3,PVD_271125_ACBS27112025160032.txt,0.9 0.8 Cổ tức (đồng) 3 3 3 2 2 Suất sinh lợi ...,2,64,"[0.019091611728072166, 0.0036112633533775806, ...","{'indices': [6, 143161, 132208, 84538, 18759, ..."
4,PVD_271125_ACBS27112025160032.txt,Cập nhật PVD – KHẢ QUAN Ngày 26/11/2025 Phòng ...,2-3,453,"[0.020067453384399414, 0.027360811829566956, -...","{'indices': [204101, 42897, 82375, 397, 46, 70..."


# Qdrant

In [ ]:
client = QdrantClient(
    url=QDRANT_CLOUD_URL,
    api_key= QDRANT_APIKEY,
)

client.create_collection(
    collection_name="hybrid_collection",
    vectors_config={
        "dense": models.VectorParams(
            size=1024,
            distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(on_disk=False),
        )
    },
)

In [28]:
import uuid

In [30]:
points = []

for _, row in df_vector.iterrows():
    point = PointStruct(
        id=str(uuid.uuid4()),   
        vector={
            "dense": row["embeddings"],
            "sparse": row["sparse"]
        },
        payload={
            "file": row["file"],
            "text": row["text"],
            "page": row["page"],
        }
    )
    points.append(point)

In [31]:
client.upsert(
    collection_name="hybrid_collection",
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

## empty cache

In [ ]:
del model
del sparse_model
import torch
torch.cuda.empty_cache()

# Query

In [3]:
client = QdrantClient(
    url=QDRANT_CLOUD_URL,
    api_key= QDRANT_APIKEY,
)

In [4]:
query_text = "Theo thông báo Tuyển sinh đào tạo từ xa trình độ đại học năm 2026 thì thời gian đào tạo được xác định dựa trên gì? Cụ thể như thế nào?"

In [13]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True,
)


In [14]:

compressed_prompt = llm_lingua.compress_prompt(
    query_text,
    rate=0.33,
    force_tokens=['\n', '?']
)
print(compressed_prompt['compressed_prompt'])
query_text = compressed_prompt['compressed_prompt']

tạo học 2026 thời gian đào tạo xác định??


In [ ]:
del llm_lingua
import torch
torch.cuda.empty_cache()

In [16]:
model = SentenceTransformer(MODEL_ID)
sparse_model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 275337.24it/s]


In [17]:
dense_query = model.encode(
    query_text,
    normalize_embeddings=True
).tolist()

sparse_output = sparse_model.encode(
    [query_text],
    return_dense=False,
    return_sparse=True,
    return_colbert_vecs=False
)

sparse_dict = sparse_output["lexical_weights"][0]

sparse_query = models.SparseVector(
    indices=[int(k) for k in sparse_dict.keys()],
    values=[float(v) for v in sparse_dict.values()]
)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [18]:
print(len(dense_query))

1024


In [19]:
results = client.query_points(
    collection_name="hybrid_collection",
    prefetch=[
        models.Prefetch(
            query=dense_query,
            using="dense",
            limit=50,
        ),
        models.Prefetch(
            query=sparse_query,
            using="sparse",
            limit=50,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    limit = 50,
)

In [20]:
for point in results.points:
    print(f"""
        file: {point.payload['file']},
        text: {point.payload['text']},
        score: {point.score}
            """)


        file: 5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,
        text: BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM TRƯỜNG ĐẠI HỌC QUY NHƠN Độc lập -Tự do - Hạnh phúc Số: /TB-ĐHQN Gia Lai, ngày tháng năm 2025 THÔNG BÁO Về thời gian đào tạo tối đa trình độ thạc sĩ của các khoá 24B, 25A, 25B, 26A Căn cứ Thông tư số 23/2021/TT-BGDĐT ngày 30/8/2021 của Bộ trưởng Bộ Giáo dục và Đào tạo ban hành Quy chế tuyển sinh và đào tạo trình độ thạc sĩ; Căn cứ Quyết định số 2705/QĐ-ĐHQN ngày 21/10/2021 của Hiệu trưởng về việc ban hành Quy chế tuyển sinh và đào tạo trình độ thạc sĩ của Trường Đại học Quy Nhơn; Căn cứ các Quyết định về việc công nhận học viên cao học các khoá 24B, 25A, 25B, 26A của Hiệu trưởng Trường Đại học Quy Nhơn, Thời gian tối đa để học viên hoàn thành khoá học không vượt quá 02 lần thời gian theo kế hoạch học tập chuẩn toàn khoá Đối chiếu với các quy định trên, thời hạn tối đa để học viên hoàn thành khóa học của các khoá 24B, 25A, 25B, 26A như

# Rerank

In [22]:
from FlagEmbedding import FlagReranker


In [23]:
reranker = FlagReranker(
    "BAAI/bge-reranker-v2-m3",
    use_fp16=True
)

In [24]:
query = query_text
documents = [p.payload["text"] for p in results.points]

pairs = [[query, passenge] for passenge in documents]

scores = reranker.compute_score(pairs)

ranked = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

top_5 = ranked[:5]

for passenge, score in top_5:
    print("Score:", score)
    print(passenge[:300])
    print("-" * 40)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Score: 1.890625
Tất cả thí sinh phải đảm bảo đủ điều kiện về sức khỏe để học tập và có hồ sơ dự tuyển đầy đủ, hợp lệ Đối với người khuyết tật, Nhà trường sẽ căn cứ vào tình trạng sức khỏe cụ thể để xem xét bố trí ngành học phù hợp 01 6 01 03 2 3 Thủ tục dự tuyển Thí sinh nộp 01 bộ hồ sơ giấy (trực tiếp hoặc bưu điệ
----------------------------------------
Score: -0.80029296875
Đối chiếu với các quy định trên, thời hạn tối đa để học viên hoàn thành khóa học của các khoá 24B, 25A, 25B, 26A như sau: TT Khoá đào tạo Thời hạn tối đa để học viên hoàn thành khóa học 1 24B (12/2021-2023) 29/12/2025 2 25A (08/2022-2024) 29/08/2026 3 25B (01/2023-2025) 06/01/2027 4 26A (07/2023-2
----------------------------------------
Score: -1.6220703125
BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM TRƯỜNG ĐẠI HỌC QUY NHƠN Độc lập -Tự do - Hạnh phúc Số: /TB-ĐHQN Gia Lai, ngày tháng năm 2025 THÔNG BÁO Về thời gian đào tạo tối đa trình độ thạc sĩ của các khoá 24B, 25A, 25

In [27]:
del reranker
import torch
torch.cuda.empty_cache()

# Prompt

In [49]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-69a04b6b-4c96cc18414f2db0225032c1;ef0f69a4-c8c4-4b89-93e8-aeb3427a3097)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

# Test

In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def query_top_n(user_query: str, df: pd.DataFrame, top_n: int = 5):
    instructional_query = f"query: {user_query}"
    
    query_vec = model.encode(instructional_query, normalize_embeddings=True).reshape(1, -1)
    
    doc_vecs = np.stack(df['embeddings'].values)
    
    scores = cosine_similarity(query_vec, doc_vecs)[0]
    
    top_indices = np.argsort(scores)[::-1][:top_n]
    
    results = []
    for idx in top_indices:
        results.append({
            "file_name": df.iloc[idx]['file'],
            "page": df.iloc[idx].get('page', 'N/A'),
            "text": df.iloc[idx]['text'],
            "score": round(float(scores[idx]), 4)
        })
    return results

results = query_top_n("Hội thảo Tham vấn về xây dựng các phương án phòng ngừa, ứng phó, bảo đảm an ninh môi trường biển của tỉnh Gia Lai được tổ chức vào thời gian và địa điểm cụ thể nào?", df_vector)

for res in results:
    print(f"[{res['score']}] File: {res['file_name']} (Trang: {res['page']})")
    print(f"Content: {res['text'][:200]}...")
    print("-" * 30)

[0.9124] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 1)
Content: 1 BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc Số: /KH-ĐHQN Gia Lai, ngày tháng 12 năm 2025 KẾ HOẠCH Tổ chức Hội thảo khoa học “Tham vấ...
------------------------------
[0.9099] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 1-2)
Content: - Ban Giám đốc Công an tỉnh, lãnh đạo, chỉ huy PV01, PC03, PA02, PA03, PA04, chỉ huy 15 Công an xã, phường ven biển, Ban chỉ huy Bộ đội Biên phòng thuộc Bộ Chỉ huy Quân sự tỉnh - Đại diện lãnh đạo Việ...
------------------------------
[0.8738] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 2-3)
Content: 4 Chương trình TT Thời gian Nội dung Người / đơn vị chủ trì thực hiện 1 13h30-14h00 Đón tiếp đại biểu, ổn định hội trường Trường Đại học Quy Nhơn + Công an tỉnh Gia Lai 2 14h00-14h10 Tuyên bố lý do, g...
------------------------------
[0.8445] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang:

# Testing extraction

In [ ]:
import pymupdf
import pandas as pd
import re
import statistics

def is_uppercase_ratio(text, threshold=0.8):
    letters = [c for c in text if c.isalpha()]
    if not letters:
        return False
    upper = [c for c in letters if c.isupper()]
    return len(upper) / len(letters) >= threshold



def parse_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)

    lines = []

    for page_num, page in enumerate(doc, start=1):
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" not in block:
                continue

            for line in block["lines"]:
                text = "".join([s["text"] for s in line["spans"]]).strip()
                if not text:
                    continue

                span = line["spans"][0]

                lines.append({
                    "page": page_num,
                    "text": text,
                    "font_size": span["size"],
                    "font": span["font"],
                    "flags": span["flags"]
                })

    body_font_size = statistics.mode([l["font_size"] for l in lines])

    results = []

    current_heading = None
    current_body = []
    current_page = None

    for line in lines:
        text = line["text"]
        is_bold = "Bold" in line["font"] or (line["flags"] & 2)

        is_heading_candidate = (
            is_bold
            or line["font_size"] > body_font_size
            or re.match(r"^\d+[\.\)]", text)
            or is_uppercase_ratio(text)
        )

        if is_heading_candidate:
            if current_heading and current_body:
                results.append({
                    "page": current_page,
                    "heading": current_heading,
                    "body": " ".join(current_body).strip()
                })

            current_heading = text
            current_body = []
            current_page = line["page"]

        else:
            if current_heading:
                current_body.append(text)

    if current_heading and current_body:
        results.append({
            "page": current_page,
            "heading": current_heading,
            "body": " ".join(current_body).strip()
        })

    return pd.DataFrame(results)

In [43]:
pdf_path =  "/home/jacktran/RAG/experiment/files/927ac79f-30b6-420c-af2f-9229ba129bdf.pdf"

df = parse_pdf(pdf_path)
df

,page,heading,body
0,1,"năm học 2025-2026, đại học chính quy",Căn cứ Thông báo số 4567/TB-ĐHQN ngày 26/12/20...
1,1,Nơi nhận:,- HT (để báo cáo); - Các Khoa (để thực hiện); ...
2,1,TS. Đinh Anh Tuấn,70 13 01


In [44]:
pdf_path =  "/home/jacktran/RAG/experiment/files/3ac90064-2e23-43cc-9e7f-6b9ea653b552.pdf"

df = parse_pdf(pdf_path)
df

,page,heading,body
0,1,"Ngôn ngữ Anh, Công tác xã hội, Kế toán, Quản l...",Trường Đại học Quy Nhơn thông báo tuyển sinh đ...
1,1,Chỉ tiêu,"1 Luật Xét kết quả học tập trung cấp, cao đẳng..."
2,1,"2. Đối tượng, điều kiện tuyển sinh",Đối tượng tuyển sinh là những người đã có bằng...
3,2,3. Thủ tục dự tuyển,Thí sinh nộp 01 bộ hồ sơ giấy (trực tiếp hoặc ...
4,2,Lai.,Hồ sơ gồm có: - Phiếu đăng ký xét tuyển; - Đối...
5,2,"4. Thời gian đào tạo, hình thức học và chuẩn đ...",Thời gian đào tạo được xác định dựa trên trình...
6,2,5. Phí tuyển sinh,Phí tuyển sinh: 400.000 đồng/thí sinh.
7,2,Thông tin liên hệ:,"- Phòng Đào tạo (bộ phận Vừa làm vừa học), Phò..."
8,3,3,Website: https://pdt.qnu.edu.vn - Phòng Kế hoạ...
9,3,Nơi nhận:,"- Hiệu trưởng (để báo cáo); - Phòng: KH-TC, CT..."
